In [1]:
import subprocess
import os 
import re
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering
from collections import Counter

In [2]:
pockets_order = pd.read_excel('pockets_order.xlsx')
total_pocks=list(pockets_order[0])

In [3]:
total_pocks

['1blx_pocket1.pdb',
 '1byg_pocket1.pdb',
 '1fmk_pocket1.pdb',
 '1fmk_pocket2.pdb',
 '1j1b_pocket1.pdb',
 '1o6l_pocket1.pdb',
 '1p4o_pocket1.pdb',
 '1qpc_pocket1.pdb',
 '1s9i_pocket1.pdb',
 '1tki_pocket1.pdb',
 '1u59_pocket1.pdb',
 '1u59_pocket2.pdb',
 '1ua2_pocket1.pdb',
 '1vzo_pocket1.pdb',
 '1wak_pocket1.pdb',
 '1x8b_pocket1.pdb',
 '1xjd_pocket1.pdb',
 '1xjd_pocket2.pdb',
 '2a2a_pocket1.pdb',
 '2ac3_pocket1.pdb',
 '2acx_pocket1.pdb',
 '2acx_pocket2.pdb',
 '2buj_pocket1.pdb',
 '2cmw_pocket1.pdb',
 '2dq7_pocket1.pdb',
 '2dq7_pocket2.pdb',
 '2eva_pocket1.pdb',
 '2f57_pocket1.pdb',
 '2fst_pocket1.pdb',
 '2h6d_pocket1.pdb',
 '2hw6_pocket1.pdb',
 '2i0e_pocket1.pdb',
 '2izr_pocket1.pdb',
 '2j0i_pocket1.pdb',
 '2jam_pocket1.pdb',
 '2jc6_pocket1.pdb',
 '2qlu_pocket1.pdb',
 '2qol_pocket1.pdb',
 '2r2p_pocket1.pdb',
 '2r4b_pocket1.pdb',
 '2r5t_pocket1.pdb',
 '2rei_pocket1.pdb',
 '2rku_pocket1.pdb',
 '2vd5_pocket1.pdb',
 '2vz6_pocket1.pdb',
 '2vz6_pocket2.pdb',
 '2w4o_pocket1.pdb',
 '2wel_pocket

In [4]:
atp_dic = {}
with open('ATP_sites.txt','r') as f:
    for line in f:
        count = 0
        for word in line.split("||"):
            if count == 0:
                name = word[:4]
                atp_dic[name] = []
            elif count == 1:
#                 pos1 = word[:3]
                atp_dic[name].append(int(word[:3]))
            elif count == 2 and word != '\n':
#                 print(word[:3])
                atp_dic[name].append(int(word[:3]))
                
            count += 1

In [5]:
atp_dic

{'1blx': [159],
 '1byg': [146],
 '1cm8': [164],
 '1fmk': [323],
 '1fvr': [170],
 '1j1b': [166],
 '1o6l': [148],
 '1p4o': [176],
 '1qpc': [152],
 '1rjb': [258],
 '1s9i': [153],
 '1u59': [152],
 '1ua2': [143],
 '1vzo': [172],
 '1xjd': [146],
 '2a2a': [161],
 '2c47': [145],
 '2cmw': [144],
 '2dq7': [148],
 '2eva': [149],
 '2f57': [593],
 '2h6d': [148],
 '2i0e': [146],
 '2j0i': [159],
 '2jam': [148],
 '2qlu': [152],
 '2r2p': [157],
 '2r4b': [168],
 '2r5t': [159],
 '2rei': [170],
 '2rku': [158],
 '2vd5': [203],
 '2vwi': [152],
 '2vz6': [162],
 '2wel': [159],
 '2x4f': [168],
 '2xik': [156],
 '2xir': [231],
 '2xrw': [169],
 '2y4i': [151],
 '2y7j': [162],
 '2ycf': [159],
 '2yex': [147],
 '2z7q': [150],
 '3a7i': [150],
 '3a99': [154],
 '3aln': [165],
 '3b2t': [181],
 '3bhh': [149],
 '3bhy': [154],
 '3brb': [167],
 '3bu3': [164],
 '3cbl': [256],
 '3cc6': [153],
 '3cok': [155],
 '3com': [139],
 '3dls': [165],
 '3dtc': [159],
 '3e7o': [161],
 '3eqc': [170],
 '3fe3': [147],
 '3fhr': [155],
 '3fxz':

In [15]:
atp_total = []

for pocket_file in total_pocks:
    positions1 = []
    with open("convertions30/"+str(pocket_file),'r') as f:
        for line in f:
            j = 2
            sentence = []
            while ( "." not in line.split()[j]):
                sentence.append(line.split()[j])
                j += 1

            positions1.append(re.sub("[^0-9]", "",sentence[-1]))
            
            
            
            
    positions1 = [int(i) for i in positions1]   

    if pocket_file[:4] in atp_dic:
        p = str(pocket_file[:4])+"_"+re.findall(r'[0-9]+', pocket_file[4:])[0]
#         print(p)
        if len(atp_dic[pocket_file[:4]]) == 1:
            if atp_dic[pocket_file[:4]][0] in positions1:
                atp_total.append([p,"atp"])
            else:
                atp_total.append([p,"non_atp"])

        if len(atp_dic[pocket_file[:4]]) == 2:
            if atp_dic[pocket_file[:4]][0] in positions1 or atp_dic[pocket_file[:4]][1] in positions1:
                atp_total.append([p,"atp"])
            else:
                atp_total.append([p,"non_atp"])
    else:
        p = str(pocket_file[:4])+"_"+re.findall(r'[0-9]+', pocket_file[4:])[0]
        atp_total.append([p,"non_atp"])

In [16]:
counter = 0
wtf = []
for pocs in atp_total:
    if pocs[1] == "atp":
        counter += 1
        wtf.append(pocs[0][:4])
print(counter)
print(len(wtf))
print(len(np.unique(wtf)))
dic_wtf = Counter(wtf)
# print(dic_wtf)

191
191
179


In [8]:
dic_wtf[max(dic_wtf, key=dic_wtf.get)]

2

In [9]:
dic_wtf["3bhh"]

1

In [17]:
atp_total_sorted = sorted(atp_total, key=lambda x:x[0])

In [18]:
atp_total_sorted

[['1blx_1', 'atp'],
 ['1byg_1', 'atp'],
 ['1fmk_1', 'atp'],
 ['1fmk_2', 'non_atp'],
 ['1j1b_1', 'atp'],
 ['1o6l_1', 'atp'],
 ['1p4o_1', 'atp'],
 ['1qpc_1', 'atp'],
 ['1s9i_1', 'atp'],
 ['1tki_1', 'non_atp'],
 ['1u59_1', 'atp'],
 ['1u59_2', 'non_atp'],
 ['1ua2_1', 'atp'],
 ['1vzo_1', 'atp'],
 ['1wak_1', 'non_atp'],
 ['1x8b_1', 'non_atp'],
 ['1xjd_1', 'atp'],
 ['1xjd_2', 'non_atp'],
 ['2a2a_1', 'atp'],
 ['2ac3_1', 'non_atp'],
 ['2acx_1', 'non_atp'],
 ['2acx_2', 'non_atp'],
 ['2buj_1', 'non_atp'],
 ['2cmw_1', 'atp'],
 ['2dq7_1', 'atp'],
 ['2dq7_2', 'atp'],
 ['2eva_1', 'atp'],
 ['2f57_1', 'atp'],
 ['2fst_1', 'non_atp'],
 ['2h6d_1', 'atp'],
 ['2hw6_1', 'non_atp'],
 ['2i0e_1', 'atp'],
 ['2izr_1', 'non_atp'],
 ['2j0i_1', 'atp'],
 ['2jam_1', 'atp'],
 ['2jc6_1', 'non_atp'],
 ['2qlu_1', 'atp'],
 ['2qol_1', 'non_atp'],
 ['2r2p_1', 'atp'],
 ['2r4b_1', 'atp'],
 ['2r5t_1', 'atp'],
 ['2rei_1', 'atp'],
 ['2rku_1', 'atp'],
 ['2vd5_1', 'atp'],
 ['2vz6_1', 'atp'],
 ['2vz6_2', 'non_atp'],
 ['2w4o_1', 'non

In [19]:
pd.DataFrame(atp_total_sorted).to_excel('atp_pockets.xlsx')